In [1]:
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
import numpy as np
import os

import torch
import torchvision



import transformers
import simpletransformers

from simpletransformers.language_modeling import LanguageModelingModel
from simpletransformers.classification import ClassificationModel


from transformers import BertForSequenceClassification, BertTokenizer, BertForMaskedLM

from sklearn.metrics.pairwise import cosine_similarity, paired_euclidean_distances
from sklearn.metrics.pairwise import euclidean_distances

from sklearn.model_selection import train_test_split


from sklearn.metrics import accuracy_score, f1_score, classification_report

from utils import *

In [3]:
df = pd.read_csv('MedicineCorpus_Final.csv',sep='\t',lineterminator='\n')

In [4]:
df = df[['id','text','PET']]

In [5]:
df['labels'] = df.PET.apply(lambda x : 1 if x == "Yes" else 0)

### Model Training

In [22]:
train_args = {
    "output_dir":'PETModel',
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    'fp16':False,
    'num_train_epochs':2
}



model = ClassificationModel('bert', 'bert-base-uncased', num_labels=2, use_cuda=True,
                            args=train_args) 

# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', output_hidden_states= False)
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

print('Model loaded')

Model loaded


In [7]:
trainDf, testDf, _, _ = train_test_split(df, df.labels, test_size = 0.2)

In [8]:
trainDf = trainDf[['text','labels']]

testDf = testDf[['text','labels']]

trainDf.head()

,text,labels
1396,I don't not study these lame lamez I'm just go...,0
1444,"Legends to Singapore, Part I:\n""Where are my A...",0
5235,@MALEFICENTHILDA lmao damn that's some crazy s...,0
3938,@thebestjasmine @maureenjohnson I TRYING TOO G...,0
4213,Someone give Keanu Reeves a Xanax. Dude has no...,0


In [25]:
trainDf_ = upsampleDf(trainDf, targetVar='labels')

In [26]:
trainDf_.labels.value_counts()

1    4349
0    4349
Name: labels, dtype: int64

In [27]:
model.train_model(trainDf_)

Running loss: 0.010134


Running loss: 0.001665



In [9]:
pred = model.eval_model(testDf)

In [10]:
pred = pred[1]

In [12]:
from scipy.special import softmax

In [14]:
softmax(pred,1)

array([[0.0016298 , 0.9983703 ],
       [0.99867266, 0.00132725],
       [0.99813825, 0.00186176],
       ...,
       [0.9979529 , 0.00204708],
       [0.9983915 , 0.00160852],
       [0.99823177, 0.00176836]], dtype=float32)

In [12]:
pred = pred[1]
finalPred = np.argmax(pred,1)

In [13]:
accuracy_score(testDf.labels, finalPred)

0.9512022630834512

In [31]:
print(classification_report(testDf.labels, finalPred)) 

              precision    recall  f1-score   support

           0       0.89      0.92      0.91      1064
           1       0.73      0.66      0.69       350

    accuracy                           0.86      1414
   macro avg       0.81      0.79      0.80      1414
weighted avg       0.85      0.86      0.85      1414



### Loading trained model

In [15]:
model = ClassificationModel('bert', 'PETModel/checkpoint-2176-epoch-2/', num_labels=2, use_cuda=True) 

### Predicting on Covid Data

In [16]:
dataFolder = '/data1/roshansk/covid_data/'

In [8]:
os.listdir(dataFolder)

['messages_cm_mar1_apr23_noRT.csv',
 'messages_cm_mar1_apr23.csv',
 'CovidTrainDf_1.csv',
 'covid19_msgs_2020_02.csv']

In [17]:
covidDf = pd.read_csv(os.path.join(dataFolder, 'messages_cm_mar1_apr23_noRT.csv'), nrows = 100000)

In [10]:
covidDf.head()

,message_id,user_id,message,created_at_utc,retweeted,retweet_message_id,in_reply_to_message_id,in_reply_to_user_id,favorite_count,retweet_count,...,tweet_location,tweet_location_short,place_type,coordinates,coordinates_state,fips,county,city,state,RT
0,1238220897720336390,790013818999074820,loving how i predicted the #coronavirus in my ...,2020-03-12 21:50:41,NaN,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,12095.0,Orange County,winter park,FL,0
1,1238220898307432448,320608440,The markets is hemorrhaging #trump and his pal...,2020-03-12 21:50:41,NaN,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,25005.0,Bristol County,Freetown,MA,0
2,1238220912530522123,305058336,Amar es prevenirn#coronavirus,2020-03-12 21:50:45,NaN,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,12086.0,Miami-Dade County,Miami,FL,0
3,1238220933384593413,137437056,"Chris's glib, histrionic commentary on COVID-1...",2020-03-12 21:50:50,NaN,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,11001.0,District of Columbia,Washington,D.C.,0
4,1238220933766230017,1012237740,If only COVID-19 started in Madagascar,2020-03-12 21:50:50,NaN,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,40015.0,Caddo County,Anadarko,OK,0


In [18]:
covidDf = covidDf[['message','message_id']]
covidDf.columns = ['text','labels']

covidDf.labels = len(covidDf)*[0]

In [14]:
covidDf.head()

,text,labels
0,loving how i predicted the #coronavirus in my ...,0
1,The markets is hemorrhaging #trump and his pal...,0
2,Amar es prevenirn#coronavirus,0
3,"Chris's glib, histrionic commentary on COVID-1...",0
4,If only COVID-19 started in Madagascar,0


In [15]:
i = 0
subDf = covidDf.iloc[i*100000:(i+1)*100000]

In [19]:
pred = model.eval_model(covidDf)

In [20]:
pred = pred[1]
finalPred = np.argmax(pred,1)


In [23]:
pd.Series(finalPred).to_csv('finalPred_0.csv',index = False, index_label = False)

In [25]:
subDf = covidDf[finalPred==1]

In [26]:
len(subDf)

2570

In [27]:
subDf.head()

,text,labels
105,Man WHAT???! This is absolutely insane lmaooooo,0
158,- Do not downplay my fears. This virus can kil...,0
245,Guys. I'm not sick. I just ran up the stairs a...,0
253,Welp...,0
262,NOOO BRO STOP ðŸ˜­ðŸ˜­,0


In [30]:
for i in range(50):
    print(subDf.iloc[i]['text'])
    print("-----------")

Man WHAT???! This is absolutely insane lmaooooo
-----------
- Do not downplay my fears. This virus can kill me.nn- Be kind if/when I don't want to engage with your "hilarious"â€¦ https://t.co/E5XIaNi3XY
-----------
Guys. I'm not sick. I just ran up the stairs and am REALLY OUT OF SHAPE.n#CoronaOutbreak
-----------
Welp...
-----------
NOOO BRO STOP ðŸ˜­ðŸ˜­
-----------
It all sucks but I have to appreciate 2020â€™s sense of Irony.nn#Bond now has PLENTY of time to die.n#FastAndFuriousÂ â€¦ https://t.co/Wtez2Pkovx
-----------
PUBLIC STATEMENT FROM @TPACtix ON COVID-19 AND THE EXTRA MEASURES IN PLACE FOR PUBLIC ATTENDING SHOWS... https://t.co/bTx5Gr6nqE
-----------
All these #Coronavirus lockdowns got me feeling like I just got one of these in the mail. https://t.co/fguRDZjAXA
-----------
The Tribeca Film Festival And â€œMulanâ€ Among The Latest Coronavirus Postponementsnn#Tribeca2020 #Tribecaâ€¦ https://t.co/RMjKZR5ALW
-----------
Ahahaha....I'm dying. ðŸ¤£ðŸ¦ n#Covid_19 #COVIDãƒ¼19 http